In [1]:
import os
import sys
import tqdm
import cv2 as cv
import gc
import numpy as np
import torch
from tqdm import tqdm

In [2]:
!pip install torcheval

  Obtaining dependency information for torcheval from https://files.pythonhosted.org/packages/e4/de/e7abc784b00de9d05999657d29187f1f7a3406ed10ecaf164de06482608f/torcheval-0.0.7-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 3.2 MB/s eta 0:00:00a 0:00:01


# Single GPU Training

In [3]:
device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [4]:
import torch
import torch.nn as nn
import lzma
import tarfile
from tqdm import tqdm
import os
from torch import nn
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import torch
from tqdm.auto import tqdm
from torch.optim.lr_scheduler import OneCycleLR
from torch.utils.tensorboard import SummaryWriter
import warnings
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import tarfile
import lzma
from tqdm import tqdm
import torch.multiprocessing as mp
from torchvision import models
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

def get_data():
    
    train_files_names = []
    with open('/kaggle/input/augmentare-1000-102-2/ip102_v1.1/train.txt','r') as f:
        train_files_names = f.readlines()

    val_files_names = []
    with open('/kaggle/input/augmentare-1000-102-2/ip102_v1.1/val.txt','r') as f:
        val_files_names = f.readlines()
    with open('/kaggle/input/augmentare-1000-102-2/ip102_v1.1/test.txt','r') as f:
        test_files_names = f.readlines()
    
    train_files_names = [x[:-1] for x in train_files_names]
    train_files_names.sort()
    
    val_files_names = [x[:-1] for x in val_files_names]
    val_files_names.sort()
    
    test_files_names = [x[:-1] for x in test_files_names]
    test_files_names.sort()
    
    train_files = [x.split(' ')[0] for x in train_files_names]
    train_labels = [int(x.split(' ')[1]) for x in train_files_names]
    train_labels_pt = torch.tensor(train_labels)
    
    test_files = [x.split(' ')[0] for x in test_files_names]
    test_labels = [int(x.split(' ')[1]) for x in test_files_names]
    test_labels_pt = torch.tensor(test_labels)
    
    val_files = [x.split(' ')[0] for x in val_files_names]
    val_labels = [int(x.split(' ')[1]) for x in val_files_names]
    val_labels_pt = torch.tensor(val_labels)

    directory = '/kaggle/input/augmentare-1000-102-2/ip102_v1.1/images/'
    files = os.listdir(directory)
    files.sort()
    
    train_images,val_images, test_images = [],[],[]
    for file in tqdm(files):
        if file in train_files:
            train_images.append(directory+file)
        elif file in val_files:
            val_images.append(directory+file)
        elif file in test_files:
            test_images.append(directory+file)

      

    return train_images,val_images,train_labels,val_labels,train_labels_pt,val_labels_pt,test_images,test_labels,test_labels_pt


def class_weights(labels):
    from collections import Counter
    c = Counter(labels)
    mc = c.most_common()
    print(mc)
    weights = []
    total_samples = len(labels)
    mc.sort(key=lambda x: x[0])
    # option 1:
    # nr of samples divided by the number of samples for each class
    for i in range(len(mc)):
        weights.append(total_samples/ mc[i][1])
        
    # option 2: 1-no_samples_per_class/total_samples
    weights = torch.tensor(weights)
    print(weights)
    return weights


def get_criterion(weights):
    
    criterion = nn.CrossEntropyLoss(weight=weights,label_smoothing=0.3)
    
    return criterion

import torch
import torch.nn as nn
import torchvision.models as models


class CustomModel(nn.Module):
    def __init__(self,sota):
        super(CustomModel, self).__init__()
        self.sota = sota
        self.sota.classifier = nn.Sequential(
            nn.Linear(960, 1280),  # Original first fully connected layer
            nn.Hardswish(),
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(1280, 102)  # New fully connected layer with 256 output features
        )
        for name, param in self.sota.named_parameters():
            if not name.startswith('features.16') and not name.startswith('classifier'):
                param.requires_grad = False
        for name, param in self.sota.named_parameters():
            print(f'{name}: requires_grad={param.requires_grad}')
        
    def forward(self,x):
        x = self.sota(x)
        return  x



def get_model():
    # Load the ResNet50 model
    model = models.mobilenet_v3_large(weights='DEFAULT')

    model = CustomModel(model)
   
    
    return model



class CustomDataset(Dataset):
    def __init__(self, image_list, label_list, transform=None):
        self.image_list = image_list
        self.label_list = label_list
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image_path = self.image_list[idx]
        label = self.label_list[idx]

        # Load the image
        image = Image.open(image_path).convert("RGB")

        # Apply transformations if any
        if self.transform:
            image = self.transform(image)

        return image, label


# Define transformations if needed (e.g., resizing, normalization, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(0,1)
])



def get_datasets(train_images,val_images,train_labels,val_labels,test_labels,test_images):
    #Create an instance of your custom dataset
    custom_dataset_train = CustomDataset(train_images, train_labels, transform)

    # Create a DataLoader to handle batching and shuffling
    data_loader_train = torch.utils.data.DataLoader(custom_dataset_train, batch_size=1024, shuffle=True)

    # Create an instance of your custom dataset
    custom_dataset_val = CustomDataset(val_images, val_labels, transform)

    # Create a DataLoader to handle batching and shuffling
    data_loader_val = torch.utils.data.DataLoader(custom_dataset_val, batch_size=1024, shuffle=False)
    
    # Create an instance of your custom dataset
    custom_dataset_test = CustomDataset(test_images, test_labels, transform)

    # Create a DataLoader to handle batching and shuffling
    data_loader_test = torch.utils.data.DataLoader(custom_dataset_test, batch_size=1024, shuffle=False)


    return  data_loader_train, data_loader_val, data_loader_test



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
train_images,val_images,train_labels,val_labels,train_labels_pt,val_labels_pt,test_images,test_labels,test_labels_pt = get_data()
model = get_model()
dataset_train,dataset_val,dataset_test = get_datasets(train_images,val_images,train_labels_pt,val_labels_pt,test_labels_pt,test_images)
weights = class_weights(train_labels)
weights.to(device)
criterion = get_criterion(weights)
criterion.to(device)
model

100%|██████████| 150653/150653 [04:31<00:00, 555.21it/s]
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-5c1a4163.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 135MB/s] 


features.0.0.weight: requires_grad=False
features.0.1.weight: requires_grad=False
features.0.1.bias: requires_grad=False
features.1.block.0.0.weight: requires_grad=False
features.1.block.0.1.weight: requires_grad=False
features.1.block.0.1.bias: requires_grad=False
features.1.block.1.0.weight: requires_grad=False
features.1.block.1.1.weight: requires_grad=False
features.1.block.1.1.bias: requires_grad=False
features.2.block.0.0.weight: requires_grad=False
features.2.block.0.1.weight: requires_grad=False
features.2.block.0.1.bias: requires_grad=False
features.2.block.1.0.weight: requires_grad=False
features.2.block.1.1.weight: requires_grad=False
features.2.block.1.1.bias: requires_grad=False
features.2.block.2.0.weight: requires_grad=False
features.2.block.2.1.weight: requires_grad=False
features.2.block.2.1.bias: requires_grad=False
features.3.block.0.0.weight: requires_grad=False
features.3.block.0.1.weight: requires_grad=False
features.3.block.0.1.bias: requires_grad=False
features.

CustomModel(
  (sota): MobileNetV3(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          )
        )
      )
      (2): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(


In [6]:
dataset_test.batch_size

1024

In [7]:
model.to(device)

CustomModel(
  (sota): MobileNetV3(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          )
        )
      )
      (2): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(


In [9]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from tqdm import auto
optimizer = optim.AdamW([
    {'params': model.sota.classifier.parameters()},
    {'params': model.sota.features[16].parameters(), 'lr': 0.001}
], lr=0.01, weight_decay=0.06)

num_epochs = 20
progress_bar = tqdm(range(len(dataset_train)*num_epochs))
# lambda1 = lambda epoch: 0.95 ** epoch
# scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)
# scheduler2 = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,mode='min',factor=0.2,patience=2,min_lr=0.0001)

  0%|          | 0/2160 [00:00<?, ?it/s]

In [10]:
def save_checkpoint(model,step_num):
        torch.save(model.state_dict(), f"/kaggle/working/mobilenetv3_large_checkpoints/mobilenetv3_large_checkpoint_{step_num}.pt")
        print(f"Checkpoint saved at step {step_num}")

In [11]:
from torcheval.metrics import MulticlassAUPRC
def evaluate(data_loader,model):
  model.eval()
  data_loader.shuffle=False
  torch.cuda.empty_cache()
  ap = MulticlassAUPRC(num_classes=102)
  ap_per_class = MulticlassAUPRC(num_classes=102, average=None)
  no_correct,no_samples = 0,0
  total_loss = 0
  total_ap = 0
  total_ap_per_class = torch.zeros(102)
  
  with torch.no_grad():
    for images_val,labels_val in data_loader:
     
      images_val = images_val.to(device)
      labels_val = labels_val.to(device)
      outputs = model(images_val)
      loss = criterion(outputs,labels_val)
        
      _, predictions = torch.max(outputs, 1)
      no_correct += (predictions == labels_val).sum().item()
      no_samples += labels_val.size(0)
      total_loss += loss
      ap.update(outputs,labels_val)
      ap_per_class.update(outputs,labels_val)
      total_ap += ap.compute()
      total_ap_per_class += ap_per_class.compute()
    
    accuracy = no_correct/no_samples
    total_loss = total_loss/len(data_loader)
    total_ap /= len(data_loader)
    total_ap_per_class /= len(data_loader)
    
    data_loader.shuffle= True
    return accuracy,total_loss,total_ap,total_ap_per_class


In [12]:
def freeze_layer(model):
    for name, param in model.sota.named_parameters():
        if name.startswith('features.16') :
            param.requires_grad = False
    for name, param in model.sota.named_parameters():
        print(f'name: {name} ; requires_grad= {param.requires_grad}')
    
    return model

In [13]:
import os

# Define the directory path
directory_path = '/kaggle/working/mobilenetv3_large_checkpoints'

# Check if the directory exists, and if not, create it
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# Now you can proceed with your operations related to the directory


In [14]:
def train(epoch_to_freeze,save_steps,print_steps,scheduler,model,dataset_train,dataset_val):
    global progress_bar
    import torch
    from torcheval.metrics import MulticlassAUPRC
    metric = MulticlassAUPRC(num_classes=102)
    scaler = torch.cuda.amp.GradScaler()
    try:
        for epoch in range(num_epochs):
          if epoch == epoch_to_freeze:
              model = freeze_layer(model)
          torch.cuda.empty_cache()
          for images, labels in dataset_train:
                model.train()
                images = images.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.autocast(device_type='cuda', dtype=torch.float16):
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()



                if progress_bar.n % print_steps == 0:
                    _, predictions = torch.max(outputs, 1)
                    no_correct = (predictions == labels).sum().item()
                    no_samples = labels.size(0)
                    accuracy = no_correct/no_samples
                    metric.update(outputs,labels)
                    ap = metric.compute()
                    print(f'Step: {progress_bar.n} Loss: {round(loss.item(),4)} Accuracy for train batch: {accuracy} AP: {ap}')

                if progress_bar.n % save_steps == 0 and progress_bar.n != 0:
                    save_checkpoint(model,progress_bar.n)

                progress_bar.update(1)

          # evaluation after each epoch
          model.eval()
          accuracy_train,loss_train,ap_t,ap_pct = evaluate(dataset_train,model)

          print(f'Epoch {epoch + 1}/{num_epochs} Accuracy for training: {accuracy_train} Loss for training: {loss_train} Average Precision Per Class: {ap_pct}')
          print(f'Mean Average Precision: {ap_t}')

          accuracy_val,loss_val,ap,ap_pc = evaluate(dataset_val,model)
          if accuracy_val > 0.99:
            save_checkpoint(model,progress_bar.n)
            break
          scheduler.step(round(loss_val.item(),2))
          print(f'Epoch {epoch + 1}/{num_epochs} Accuracy for validation: {accuracy_val} Loss for validation: {loss_val} Average Precision Per Class: {ap_pc}')
          print(f'Mean Average Precision: {ap}')

    except KeyboardInterrupt as e:
        save_checkpoint(model,progress_bar.n)

In [ ]:
train(1,540,27,scheduler,model,dataset_train,dataset_val)

  0%|          | 1/2160 [00:20<12:11:54, 20.34s/it]

Step: 0 Loss: 4.6902 Accuracy for train batch: 0.015625 AP: 0.01874570921063423


  1%|▏         | 28/2160 [03:42<4:23:47,  7.42s/it]

Step: 27 Loss: 3.5847 Accuracy for train batch: 0.404296875 AP: 0.19655732810497284


  3%|▎         | 55/2160 [07:04<4:34:27,  7.82s/it]

...

  5%|▌         | 108/2160 [13:51<4:03:51,  7.13s/it]

Epoch 1/20 Accuracy for training: 0.6145797443104543 Loss for training: 3.102942943572998 Average Precision Per Class: tensor([0.4162, 0.2581, 0.3503, 0.3065, 0.5313, 0.5670, 0.4861, 0.2751, 0.2731,
        0.3635, 0.4505, 0.4011, 0.6512, 0.4303, 0.8009, 0.8768, 0.7350, 0.5504,
        0.2818, 0.3291, 0.3158, 0.5746, 0.4350, 0.3012, 0.4325, 0.8397, 0.6627,
        0.2397, 0.4204, 0.2376, 0.7494, 0.6136, 0.4882, 0.5790, 0.5934, 0.6436,
        0.6624, 0.6057, 0.2150, 0.1769, 0.4992, 0.5960, 0.5532, 0.8698, 0.3636,
        0.2145, 0.2521, 0.3848, 0.7393, 0.5404, 0.5670, 0.4900, 0.3098, 0.7711,
        0.1414, 0.5954, 0.9488, 0.4737, 0.7262, 0.6347, 0.6517, 0.8455, 0.9308,
        0.5031, 0.6087, 0.4652, 0.6552, 0.7131, 0.6561, 0.4841, 0.5404, 0.6678,
        0.6263, 0.9041, 0.7207, 0.8395, 0.5619, 0.6228, 0.8224, 0.7443, 0.9032,
        0.6550, 0.7755, 0.6715, 0.5648, 0.5010, 0.2710, 0.7540, 0.6903, 0.5360,
        0.5196, 0.5839, 0.4150, 0.3584, 0.6899, 0.6618, 0.6696, 0.6053, 0.6086,
 

  5%|▌         | 109/2160 [27:56<147:22:10, 258.67s/it]

Step: 108 Loss: 2.8784 Accuracy for train batch: 0.7294921875 AP: 0.40050429105758667


  6%|▌         | 129/2160 [29:29<2:46:16,  4.91s/it]   

In [32]:
accuracy_val,loss_val,ap,ap_pc = evaluate(dataset_test,model)

In [ ]:
accuracy_val,loss_val,ap

In [ ]:
ap_pc

In [ ]:
torch.cuda.empty_cache()